# Part I. ETL Pipeline for Pre-Processing the Files

## RE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding portion of project. 

##   The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

session = cluster.connect()    

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Create tables, queries to extract the required data 

### 1. Extract the artist, song title and song's length.
  - table name : session_music
  - Primary key : sessionID, itemINsession
  - Columns : sartist_name, song_name, song_length

#### We will extract song data by sessionId and itemInSession. Conposition of sessionId and itemInSession makes all data unique. 
#### Data will be selected at [sessionId = 338, and itemInSession  = 4]

In [8]:
query = "drop table if exists session_music"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "CREATE TABLE IF NOT EXISTS session_music "
query = query + "(sessionId int, itemInSession int, artist_name text, song_name text, song_length float, PRIMARY KEY(sessionID, itemINsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO session_music (sessionId, itemInSession, artist_name, song_name, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5]) ))

In [10]:

query = "select artist_name, song_name, song_length from session_music WHERE sessionID = 338 AND itemInSession = 4 "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print ('artist_name:', row.artist_name,'song_name:', row.song_name, 'song_length:', row.song_length,)

artist_name: Faithless song_name: Music Matters (Mark Knight Dub) song_length: 495.30731201171875


### 2. Extract artist name,  song tilte, and user name(first name,last name) length
  - table name : user_music
  - Primary key : sessionID, userID
  - Columns : itemInSession, artist_name, song_name, first_name, last_name

#### We will extract song data by sessionID and userID. Sometimes, users hear several songs during one session. So, itemInSession is necessary for primary key composition as clustering key.
#### Data will be selected at [sessionID = 182 AND userID = 10]

In [11]:
query = "drop table if exists user_music"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "CREATE TABLE IF NOT EXISTS user_music"
query = query + "(sessionID int, userID int, itemInSession int, artist_name text, song_name text, first_name text, last_name text,  PRIMARY KEY((sessionID), userID, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_music(sessionID, userID, itemInSession, artist_name, song_name, first_name, last_name)"
        query = query + " VALUES (%s, %s,%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
query = "select artist_name,  song_name, first_name, last_name from user_music WHERE sessionID = 182 AND userID = 10 "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print ( 'artist_name:', row.artist_name, 'song_name:', row.song_name, 'first_name:', row.first_name, 'last_name:', row.last_name,)
                    

artist_name: Down To The Bone song_name: Keep On Keepin' On first_name: Sylvie last_name: Cruz
artist_name: Three Drives song_name: Greece 2000 first_name: Sylvie last_name: Cruz
artist_name: Sebastien Tellier song_name: Kilometer first_name: Sylvie last_name: Cruz
artist_name: Lonnie Gordon song_name: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) first_name: Sylvie last_name: Cruz


### 3. Extract user name(first name,last name) length at [song_name is 'All Hands Against His Own']
  - table name : song_heard_user
  - Primary key : song_name,sessionID
  - Columns : itemInSession, first_name, last_name

#### We will extract user data by song_name. So, composition of song_name and userId is unique for identifying user_name who listened the target song.
#### Data will be selected at [song_name is 'All Hands Against His Own']

In [14]:

query = "drop table if exists song_heard_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "CREATE TABLE IF NOT EXISTS song_heard_user"
query = query + "(song_name text, userId int, first_name text, last_name text, PRIMARY KEY(song_name, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO song_heard_user(song_name, userId,  first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]),  line[1], line[4]))

In [16]:
query = "select  first_name, last_name from song_heard_user WHERE song_name = 'All Hands Against His Own' "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print ('first_name:', row.first_name, 'last_name:', row.last_name,)

first_name: Jacqueline last_name: Lynch
first_name: Tegan last_name: Levine
first_name: Sara last_name: Johnson


### Drop the tables before closing out the sessions

In [17]:
query = "drop table if exists session_music"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [18]:
query = "drop table if exists user_music"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [19]:
query = "drop table if exists song_heard_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()